ソース：https://qiita.com/izaki_shin/items/2b4573ee7fbea5ec8ed6

### **Install package**

In [1]:
# HuggingFace Transformers 설치
# - transformers : 주요 모듈(모델이나 토크나이저)
# - datasets : HuggingFace로 데이터셋을 다루기 위한 모듈
# cf. https://huggingface.co/docs/transformers/installation
! pip install transformers datasets

#  도호쿠 대학 일본어용 BERT 사용에 필요한 패키지 설치
! pip install fugashi ipadic

# 기타 패키지
! pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 34.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=f124652e0f5f24f5bfd527af5ded749767b58065fc41c13c446e27bc84332a2f
  Stored in directory: /root/.cache/pip

### **Download dataset**

In [2]:
!wget https://github.com/ids-cv/wrime/raw/master/wrime-ver1.tsv

--2024-04-20 07:01:10--  https://github.com/ids-cv/wrime/raw/master/wrime-ver1.tsv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver1.tsv [following]
--2024-04-20 07:01:10--  https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver1.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9487235 (9.0M) [text/plain]
Saving to: ‘wrime-ver1.tsv’

wrime-ver1.tsv      100%[===================>]   9.05M  --.-KB/s    in 0.08s   

2024-04-20 07:01:11 (114 MB/s) - ‘wrime-ver1.tsv’ saved [9487235/9487235]



In [3]:
import pandas as pd
df_wrime = pd.read_table('wrime-ver1.tsv')
df_wrime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43200 entries, 0 to 43199
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Sentence                   43200 non-null  object
 1   UserID                     43200 non-null  int64 
 2   Datetime                   43200 non-null  object
 3   Train/Dev/Test             43200 non-null  object
 4   Writer_Joy                 43200 non-null  int64 
 5   Writer_Sadness             43200 non-null  int64 
 6   Writer_Anticipation        43200 non-null  int64 
 7   Writer_Surprise            43200 non-null  int64 
 8   Writer_Anger               43200 non-null  int64 
 9   Writer_Fear                43200 non-null  int64 
 10  Writer_Disgust             43200 non-null  int64 
 11  Writer_Trust               43200 non-null  int64 
 12  Reader1_Joy                43200 non-null  int64 
 13  Reader1_Sadness            43200 non-null  int64 
 14  Reader

### **전처리**

In [4]:
# 객관감정 중 기대, 신뢰 열 제거
columns_to_drop = ['Avg. Readers_Anticipation', 'Avg. Readers_Trust']
df_wrime = df_wrime.drop(columns_to_drop, axis=1)
df_wrime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43200 entries, 0 to 43199
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Sentence               43200 non-null  object
 1   UserID                 43200 non-null  int64 
 2   Datetime               43200 non-null  object
 3   Train/Dev/Test         43200 non-null  object
 4   Writer_Joy             43200 non-null  int64 
 5   Writer_Sadness         43200 non-null  int64 
 6   Writer_Anticipation    43200 non-null  int64 
 7   Writer_Surprise        43200 non-null  int64 
 8   Writer_Anger           43200 non-null  int64 
 9   Writer_Fear            43200 non-null  int64 
 10  Writer_Disgust         43200 non-null  int64 
 11  Writer_Trust           43200 non-null  int64 
 12  Reader1_Joy            43200 non-null  int64 
 13  Reader1_Sadness        43200 non-null  int64 
 14  Reader1_Anticipation   43200 non-null  int64 
 15  Reader1_Surprise   

감정 강도가 낮은 샘플을 제외

In [5]:
# 6개의 기본 감정
emotion_names = ['Joy', 'Sadness', 'Surprise', 'Anger', 'Fear','Disgust',]
emotion_names_jp = ['喜び', '悲しみ', '驚き', '怒り', '恐れ', '嫌悪']

# 객관감정 평균（"Avg. Readers_*"）의 값을 list화 하여、새로운 열로 정의
df_wrime['readers_emotion_intensities'] = df_wrime.apply(lambda x: [x['Avg. Readers_' + name] for name in emotion_names], axis=1)


In [6]:
# 감정 강도가 낮은 샘플 제거 (강도 2 이상만 남김)
is_target = df_wrime['readers_emotion_intensities'].map(lambda x: max(x) >= 2)

df_wrime_target = df_wrime[is_target]

In [7]:
df_wrime_target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14321 entries, 0 to 43199
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Sentence                     14321 non-null  object
 1   UserID                       14321 non-null  int64 
 2   Datetime                     14321 non-null  object
 3   Train/Dev/Test               14321 non-null  object
 4   Writer_Joy                   14321 non-null  int64 
 5   Writer_Sadness               14321 non-null  int64 
 6   Writer_Anticipation          14321 non-null  int64 
 7   Writer_Surprise              14321 non-null  int64 
 8   Writer_Anger                 14321 non-null  int64 
 9   Writer_Fear                  14321 non-null  int64 
 10  Writer_Disgust               14321 non-null  int64 
 11  Writer_Trust                 14321 non-null  int64 
 12  Reader1_Joy                  14321 non-null  int64 
 13  Reader1_Sadness              14321 n

훈련/테스트셋 분리

In [8]:
df_groups = df_wrime_target.groupby('Train/Dev/Test')
df_train = df_groups.get_group('train')
df_test = pd.concat([df_groups.get_group('dev'), df_groups.get_group('test')])
print('train :', len(df_train))  # train : 13490
print('test :', len(df_test))    # test : 831

train : 13490
test : 831




---



### **Train**

東北大学の乾研究室 공개
- bert-base-japanese-whole-word-masking
- 훈련된 모델을 Hugging Face 서버에서 다운로드하여 지정한 클래스 수에 따른 모델을 생성

In [16]:
#훈련된 모델 불러오기

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 사용하려는 모델을 지정해、토크나이저와 모델을 읽어옴
checkpoint = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from datasets import Dataset
import numpy as np

# 1. Transformers용 데이터셋 형식으로 변환
# pandas.DataFrame -> datasets.Dataset
target_columns = ['Sentence', 'readers_emotion_intensities']
train_dataset = Dataset.from_pandas(df_train[target_columns])
test_dataset = Dataset.from_pandas(df_test[target_columns])

# 2. Tokenizer적용(모델 입력을 위한 전처리)
def tokenize_function(batch):
    """ 감정강도의 정규화 동시 실시 """
    tokenized_batch = tokenizer(batch['Sentence'], truncation=True, padding='max_length')
    tokenized_batch['labels'] = [x / np.sum(x) for x in batch['readers_emotion_intensities']]  # 총계=1로 정규화
    return tokenized_batch

train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
test_tokenized_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/13490 [00:00<?, ? examples/s]

Map:   0%|          | 0/831 [00:00<?, ? examples/s]

In [18]:
#훈련
import accelerate
from transformers import TrainingArguments, Trainer
from datasets import load_metric

# 평가 지표 정의
# https://huggingface.co/docs/transformers/training
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    label_ids = np.argmax(labels, axis=-1)
    return metric.compute(predictions=predictions, references=label_ids)

# 훈련 시의 설정
# https://huggingface.co/docs/transformers/v4.21.1/en/main_classes/trainer#transformers.TrainingArguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=8,
    num_train_epochs=2.0,
    evaluation_strategy="steps", eval_steps=200)  # 200단계마다 테스트 데이터로 평가
# Trainer를 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    compute_metrics=compute_metrics,
)

# 훈련 실행
trainer.train()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable

Step,Training Loss,Validation Loss,Accuracy
200,No log,0.298814,0.642599
400,No log,0.263912,0.714801
600,0.347600,0.247771,0.738869
800,0.347600,0.249790,0.719615
1000,0.303100,0.232354,0.759326
1200,0.303100,0.227536,0.774970
1400,0.303100,0.234022,0.758123
1600,0.285100,0.218786,0.779783
1800,0.285100,0.226057,0.768953
2000,0.255200,0.221624,0.776173


TrainOutput(global_step=3374, training_loss=0.27087727961062535, metrics={'train_runtime': 2981.9763, 'train_samples_per_second': 9.048, 'train_steps_per_second': 1.131, 'total_flos': 7098991220367360.0, 'train_loss': 0.27087727961062535, 'epoch': 2.0})

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




---



### **Evaluate**

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

def np_softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

def analyze_emotion(text, show_fig=False):
    # 推論モードを有効化
    model.eval()

    # 入力データ変換 + 推論
    tokens = tokenizer(text, truncation=True, return_tensors="pt")
    tokens.to(model.device)
    preds = model(**tokens)
    prob = np_softmax(preds.logits.cpu().detach().numpy()[0])
    out_dict = {n: p for n, p in zip(emotion_names_jp, prob)}

    # 棒グラフを描画
    if show_fig:
        plt.figure(figsize=(8, 3))
        df = pd.DataFrame(out_dict.items(), columns=['name', 'prob'])
        sns.barplot(x='name', y='prob', data=df)
        plt.title('入力文 : ' + text, fontsize=15)
    else:
        print(out_dict)

In [22]:
analyze_emotion('今日から長期休暇だぁーーー！！！')

{'喜び': 0.99873585, '悲しみ': 0.00022805009, '驚き': 0.00069270225, '怒り': 7.318467e-05, '恐れ': 0.00013261379, '嫌悪': 0.00013760087}
